# CSE2050 Homework 3
**Name: Joshua Hartzfeld**  | Topic: Web Scraping

#### Took me longer than i wanted but it is done ( to my extent of testing)
* First cell is imports plus the URL defined globally because its not going to change anytime soon


In [4]:
import bs4
import urllib
import dateparser

URL = "https://www.fit.edu/clemente-center/fitness-schedule/"


* This cell is the function is_open() this function is in charge of parsing dates from the day_table that is scraped from the website in get_schedule_table() if it does find someting it prints what its found, and returns otherwise it prints that the center is not open and lists the times


In [5]:
def is_open(input_date, day_table, open_hours):
    for i in range(1, len(day_table)):
        time = day_table[i][0]
        times = time.split('-')
        lower_time = dateparser.parse(times[0],
                                      settings={'RELATIVE_BASE': input_date})
        upper_time = dateparser.parse(times[1],
                                      settings={'RELATIVE_BASE': input_date})
        if lower_time <= input_date <= upper_time:
            print(f'{day_table[i][1]}' + " is held in " + f'{day_table[i][3]}' +
                  " from " + f'{lower_time:%m/%d/%Y %I:%M%p}' + " - " +
                  f'{upper_time:%m/%d/%Y %I:%M%p}')
            return 0
    print("Clemente Center is not open on " + f'{input_date:%m/%d/%Y %I:%M%p}.' +
          "The hours are")
    print(
        f'{open_hours[0]}-{open_hours[1]} {open_hours[2]}-{open_hours[3]}\n{open_hours[4]} {open_hours[5]}-{open_hours[6]}\n{open_hours[7]} {open_hours[8]}-{open_hours[9]}'
    )
    return 0



* This method like told above scrapes a specific table from the site using the date keyword that is above it i.e "Monday" "Tuesday" etc. This can be done because of the find_next() functionality of dateparser. After the table is grabbed a funky litte for loop scrapes together a 2D list of non empty values from the table

In [ ]:
def get_schedule_table(date):
    table_data = []
    response = urllib.request.urlopen(URL)
    doc = bs4.BeautifulSoup(response, features="html5lib")
    date_location = doc.find(id="page-content").find(
        class_="twelve wide computer column").find(string=date)
    table = date_location.find_next("table")

    rows = table.find_all("tr")
    for row in rows:
        col = row.find_all('td')
        col = [ele.text.strip() for ele in col]
        table_data.append([ele for ele in col if ele])

    return table_data



* The method get_opening_hours() scrapes the side column to get the centers opening times. This is used when the default print case is met

In [6]:
def get_opening_hours():
    response = urllib.request.urlopen(URL)
    doc = bs4.BeautifulSoup(response)
    sidebar_content = doc.find(id='page-content').find(
        class_='four wide computer column sidebar')
    i = 1
    for item in sidebar_content.find_all("p"):
        if (i == 6):
            hours = (item.get_text())
        i += 1

    hours = hours.strip("Fall 2022\xa0")
    hours = hours.replace(':', '')
    hours = hours.replace('pm', ' ')
    hours = hours.replace('-', ' ')
    hours = hours.split()
    for i in range(3, len(hours), 3):
        hours[i] += "pm"
        if i == 9:
            hours[i - 1] += "pm"
    return hours


* The main methods main job is to take input the input is split into 2 things, the date which is used to compare dates to find classes and datef which is the formatted version mainly used as a keyword to find the very next table on the website in get_schedule_table() 

In [7]:
def main():
    # input_date = '9/27/2022 11:30am' <- at time of testing it prints Group Cycling is held in Grp Fitness Rm from 
    # 09/27/2022 11:00AM - 09/27/2022 11:50AM date parser is kinda weird with relative dates and such and the website
    # looks like its changed so i did all of my testing from the florida tech website using my birthday 

    input_date = input("Enter a date and time format-(mm/dd/yyyy hh:mm am/pm): ")
    date = dateparser.parse(input_date)
    datef = f'{date:%A}'

    open_hours = get_opening_hours()
    day_table = get_schedule_table(datef)
    is_open(date, day_table, open_hours)
    return 0


main()


Enter a date and time format-(mm/dd/yyyy hh:mm am/pm): 9/27/2022 11:30am
Group Cycling is held in Grp Fitness Rm from 09/27/2022 11:00AM - 09/27/2022 11:50AM


0